Simple example of a homogeneous ensemble using learning networks

In [1]:
using MLJ

learning network (composite model spec):

In [2]:
Xs = source()
ys = source(kind=:target)

atom = @load DecisionTreeRegressor
atom.n_subfeatures = 4

machines = (machine(atom, Xs, ys) for i in 1:100)

Base.Generator{UnitRange{Int64},getfield(Main.##367, Symbol("##1#2"))}(getfield(Main.##367, Symbol("##1#2"))(), 1:100)

overload summation for nodes:

In [3]:
Base.sum(v...) = sum(v)
Base.sum(v::AbstractVector{<:AbstractNode}) = node(sum, v...)

yhat = sum([predict(m, Xs) for  m in machines]);

new composite model type and instance:

In [4]:
one_hundred_trees = @from_network OneHundredTrees(atom=atom) <= yhat

X, y = @load_boston;
evaluate(one_hundred_trees, X, y, measure=rms)

Evaluating over 6 folds: 100%[=========================] Time: 0:00:01


(measure = MLJBase.RMS[rms],
 measurement = [2329.62],
 per_fold = Array{Float64,1}[[2313.73, 2288.75, 2703.03, 2769.71, 1844.1, 1895.41]],
 per_observation = Missing[missing],)

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*